## Question 3

In [15]:
import tweepy
from tweepy import OAuthHandler

consumer_key = 'aZgjP0NZeM4iiBaQXJcyOKLJB'
consumer_secret = 'VRLyop5JLP8Kh91CMnE0rpOKUhKEtrrVTCOozHVOd8QEFx69XX'
access_token = '326266960-9RZItG8Q4FUER0BOX4eO8mrvsvrrDWzaWiJqoens'
access_secret = 'v6UHn9puTUv5OgcDwGqf3UxfCZv6XSJZ1gB8UkXOFrnQo'

auth = OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_secret)

api = tweepy.API(auth)

In [21]:
for status in tweepy.Cursor(api.).items(1):
    #Process a signle status
    print(status._json['user'])
    print("####")
    print(status.text)
    print("####")

{'id': 81994188, 'id_str': '81994188', 'name': 'Producer 9-0 says #ThxMLK', 'screen_name': 'nine_oh', 'location': 'Kansas City, MO', 'description': '#VolkswagenBeliever CEO 9.0 Mag, 9.0 The Underground, #Bogardthat No Limit East, Booking ☛#8186499660 ❤️God #CowboysNation #Lakers #Twittergoons🎤 Chevy Suxs', 'url': 'https://t.co/20OUcBabG3', 'entities': {'url': {'urls': [{'url': 'https://t.co/20OUcBabG3', 'expanded_url': 'http://www.nineoh.us', 'display_url': 'nineoh.us', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 1073787, 'friends_count': 1029304, 'listed_count': 2397, 'created_at': 'Tue Oct 13 02:47:15 +0000 2009', 'favourites_count': 101, 'utc_offset': -21600, 'time_zone': 'America/Chicago', 'geo_enabled': False, 'verified': False, 'statuses_count': 1534103, 'lang': 'en', 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': '000000', 'profile_background_image_url': 'http://a

In [22]:
import json

my_timeline = tweepy.Cursor(api.home_timeline).items(50)

with open("tweets.json", "a") as outfile:  
    for tweet in my_timeline:
        json.dump(tweet._json, outfile)
        
